In [129]:
import os
from dotenv import load_dotenv
from collections import OrderedDict
import pandas as pd
import numpy as np

import chart_studio
import chart_studio.plotly as py
import plotly
import plotly.graph_objects as go
import plotly.express as px

import hijri_converter
from hijri_converter import Hijri, Gregorian, convert

In [130]:
# Load environment variables from .env file
load_dotenv()
plotly_chart_studio_username = os.environ['PLOTLY_USERNAME']
plotly_chart_studio_api_key = os.environ['PLOTLY_API_KEY']

# set up connection to chart studio account
chart_studio.tools.set_credentials_file(username=plotly_chart_studio_username, api_key=plotly_chart_studio_api_key)
chart_studio.tools.set_config_file(world_readable=True, sharing='public')

# other global variables
UPLOAD_TO_CHART_STUDIO = True
TOURISM_COMPANY_NAME = "test company"
HIJRI_YEAR = 1443

In [131]:
daily_rows_weekly_df = pd.read_excel(f"./weekly_target_dates_accum_df_{HIJRI_YEAR}.xlsx", sheet_name="Sheet1", converters={'Reservation Creation Date' : lambda x:Hijri.fromisoformat(x)})
daily_rows_weekly_df.head(10)

,Reservation Creation Date,W1 - Ramadhan,W2 - Ramadhan,W3 - Ramadhan,W4 - Ramadhan,W1 - Shawwal,W2 - Shawwal,W3 - Shawwal,W4 - Shawwal,W1 - Dhu al-Qi’dah,W2 - Dhu al-Qi’dah,W3 - Dhu al-Qi’dah,W4 - Dhu al-Qi’dah,W1 - Dhu al-Hijjah,W2 - Dhu al-Hijjah,W3 - Dhu al-Hijjah,W4 - Dhu al-Hijjah
0,1443-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1443-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1443-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1443-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1443-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1443-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1443-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1443-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1443-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1443-01-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [132]:
def display_full_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)
        
def hijri_to_greg(hijri_date):
    dt_greg = hijri_date.to_gregorian().ctime()
    dt_greg = pd.to_datetime(dt_greg)
    return dt_greg

def greg_to_hijri(greg_date):
    hijri_date = Gregorian(greg_date.year, greg_date.month, greg_date.day)
    hijri_date = hijri_date.to_hijri()
    return hijri_date

def increment_hijri(hijri_date, dateOffset=1):
    dt_greg = hijri_to_greg(hijri_date)
    dt_greg += pd.DateOffset(days=dateOffset)
    incremented_hijri_date = Gregorian(dt_greg.year, dt_greg.month, dt_greg.day).to_hijri()
    return incremented_hijri_date

def to_hijri_month_comma_year(hijri_date):
    return f"{hijri_date.month_name()}, {hijri_date.year}"

def save_fig_to_chart_studio(fig, filename, create_copy=True):
    if not UPLOAD_TO_CHART_STUDIO:
        return
    if create_copy:
        fig = go.Figure(fig)
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0), # Set margins to 0
        autosize=True, # makes plot auto resize if embedded as a frame in another application based on the application's defined width/height of the frame
    )  
    filename = f"{TOURISM_COMPANY_NAME} - {HIJRI_YEAR} - {filename}"
    py.plot(fig, filename=filename, auto_open=False)

In [133]:
def divide_hijri_weeks(year):
    global target_start_day_hijri, target_end_day_hijri
    weeks_to_hijri_start_and_end_dates = {}
    week_start_day = Hijri(year, target_start_day_hijri.month, target_start_day_hijri.day) 
    cur_day = Hijri(year, target_start_day_hijri.month, target_start_day_hijri.day)
    i = 1
    while cur_day <= target_end_day_hijri:
        if i % 7 == 0:
            weeks_to_hijri_start_and_end_dates[i//7] = f"{week_start_day} to {cur_day}"
            week_start_day = increment_hijri(cur_day)
        cur_day = increment_hijri(cur_day)
        i += 1
    return weeks_to_hijri_start_and_end_dates

In [134]:
# divide_hijri_weeks(1441)

In [135]:
daily_rows_weekly_df.insert(1, 'Reservation Creation Date (Greg)', daily_rows_weekly_df['Reservation Creation Date'].apply(lambda x: hijri_to_greg(x)))
daily_rows_weekly_df.insert(2, 'Hijri Month', daily_rows_weekly_df['Reservation Creation Date'].apply(to_hijri_month_comma_year))

In [136]:
daily_rows_weekly_df.head()

,Reservation Creation Date,Reservation Creation Date (Greg),Hijri Month,W1 - Ramadhan,W2 - Ramadhan,W3 - Ramadhan,W4 - Ramadhan,W1 - Shawwal,W2 - Shawwal,W3 - Shawwal,W4 - Shawwal,W1 - Dhu al-Qi’dah,W2 - Dhu al-Qi’dah,W3 - Dhu al-Qi’dah,W4 - Dhu al-Qi’dah,W1 - Dhu al-Hijjah,W2 - Dhu al-Hijjah,W3 - Dhu al-Hijjah,W4 - Dhu al-Hijjah
0,1443-01-01,2021-08-09,"Muharram, 1443",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1443-01-02,2021-08-10,"Muharram, 1443",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1443-01-03,2021-08-11,"Muharram, 1443",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1443-01-04,2021-08-12,"Muharram, 1443",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1443-01-05,2021-08-13,"Muharram, 1443",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
greg_daily_rows_weekly_df = daily_rows_weekly_df.set_index('Reservation Creation Date (Greg)', inplace=False)
greg_daily_rows_weekly_df.drop(columns=greg_daily_rows_weekly_df.columns[:2], inplace=True)

hijri_daily_rows_weekly_df = daily_rows_weekly_df.set_index('Reservation Creation Date', inplace=False)
hijri_daily_rows_weekly_df.drop(columns=hijri_daily_rows_weekly_df.columns[:2], inplace=True)

In [138]:
greg_daily_rows_weekly_df.sum()


W1 - Ramadhan         331234.0
W2 - Ramadhan         281227.0
W3 - Ramadhan         240115.0
W4 - Ramadhan         226902.0
W1 - Shawwal          404467.0
W2 - Shawwal          583079.0
W3 - Shawwal          447167.0
W4 - Shawwal          406594.0
W1 - Dhu al-Qi’dah    212664.0
W2 - Dhu al-Qi’dah    208532.0
W3 - Dhu al-Qi’dah    243813.0
W4 - Dhu al-Qi’dah    377326.0
W1 - Dhu al-Hijjah    251272.0
W2 - Dhu al-Hijjah    173285.0
W3 - Dhu al-Hijjah    671928.0
W4 - Dhu al-Hijjah    332591.0
dtype: float64

In [139]:
Hijri(1444, 9, 1).to_gregorian()

Gregorian(2023, 3, 23)

In [140]:
px.line(greg_daily_rows_weekly_df)

In [145]:
display_full_df(hijri_daily_rows_weekly_df['W4 - Ramadhan'])

Reservation Creation Date
1443-01-01       0.0
1443-01-02       0.0
1443-01-03       0.0
1443-01-04       0.0
1443-01-05       0.0
1443-01-06       0.0
1443-01-07       0.0
1443-01-08       0.0
1443-01-09       0.0
1443-01-10       0.0
1443-01-11       0.0
1443-01-12       0.0
1443-01-13       0.0
1443-01-14       0.0
1443-01-15       0.0
1443-01-16       0.0
1443-01-17       0.0
1443-01-18       0.0
1443-01-19       0.0
1443-01-20       0.0
1443-01-21       0.0
1443-01-22       0.0
1443-01-23       0.0
1443-01-24       0.0
1443-01-25       0.0
1443-01-26       0.0
1443-01-27       0.0
1443-01-28       0.0
1443-01-29       0.0
1443-01-30       0.0
1443-02-01       0.0
1443-02-02       0.0
1443-02-03       0.0
1443-02-04       0.0
1443-02-05       0.0
1443-02-06       0.0
1443-02-07       0.0
1443-02-08       0.0
1443-02-09       0.0
1443-02-10       0.0
1443-02-11       0.0
1443-02-12       0.0
1443-02-13       0.0
1443-02-14       0.0
1443-02-15       0.0
1443-02-16       0.0
1443-02-